In [1]:
import os
import numpy as np

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
from tensorflow.python.keras.models import *
from tensorflow.python.keras.utils.generic_utils import get_custom_objects
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau

# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def dice_coef(y_true, y_pred, smooth=0.001):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


get_custom_objects().update({

    'dice_coef' : dice_coef,
    'dice_coef_loss' : dice_coef_loss,
        
})


# load models from file
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = '../result/model_save/A2C_A4C_Deep_MVAU_Net_CV_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

# make a prediction with a model
def predict_model(model, inputX):
    
    return model.predict(inputX, batch_size=4, verbose=1)

def prediction_image(pr_1, pr_2, pr_3, pr_4):
    
    fn_prediction = (pr_1 + pr_2 + pr_3 + pr_4) / 4
    
    return fn_prediction

In [3]:
# load test dataset

test_image = np.load('../data/test_dataset/test_image.npy').astype(np.float32)
test_label = np.load('../data/test_dataset/test_label.npy').astype(np.float32)

In [4]:
# Normalization

if test_image.max() == 255:
    
    test_image = test_image / 255
    
if test_label.max() == 255:
    
    test_label = test_label / 255
    
print(test_image.min(), test_image.max())
print(test_label.min(), test_label.max())

0.0 1.0
0.0 1.0


In [5]:
cv_num = 4
models = load_all_models(cv_num)

2021-12-07 21:14:55.412224: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-12-07 21:14:55.497764: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: Tesla P100-SXM2-16GB computeCapability: 6.0
coreClock: 1.4805GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-12-07 21:14:55.497957: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-12-07 21:14:55.499810: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-12-07 21:14:55.501477: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-12-07 21:14:55.501745: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.

>loaded ../result/model_save/A2C_A4C_Deep_MVAU_Net_CV_1.h5
>loaded ../result/model_save/A2C_A4C_Deep_MVAU_Net_CV_2.h5
>loaded ../result/model_save/A2C_A4C_Deep_MVAU_Net_CV_3.h5
>loaded ../result/model_save/A2C_A4C_Deep_MVAU_Net_CV_4.h5


In [6]:
_, prediction_1 = models[0].predict(test_image, batch_size=1, verbose=1)
_, prediction_2 = models[1].predict(test_image, batch_size=1, verbose=1)
_, prediction_3 = models[2].predict(test_image, batch_size=1, verbose=1)
_, prediction_4 = models[3].predict(test_image, batch_size=1, verbose=1)

2021-12-07 21:15:34.085777: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-12-07 21:15:34.545515: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2021-12-07 21:15:36.056358: W tensorflow/stream_executor/gpu/redzone_allocator.cc:312] Not found: ./bin/ptxas not found
Relying on driver to perform ptx compilation. This message will be only logged once.


100/100 [==============================] - 5s 52ms/sample


In [7]:
# make a image from probability

threshold_value = 0.5

prediction_1 = (prediction_1 > threshold_value) * 1.0
prediction_2 = (prediction_2 > threshold_value) * 1.0
prediction_3 = (prediction_3 > threshold_value) * 1.0
prediction_4 = (prediction_4 > threshold_value) * 1.0

pred_label = prediction_image(prediction_1, prediction_2, prediction_3, prediction_4)

In [8]:
# Evaluation

a2c_dice_list = []
a4c_dice_list = []

a2c_jac_list = []
a4c_jac_list = []

for i in range(test_label.shape[0]):

############### A2C ################
    
    a2c_seg1 = pred_label[i,:,:,0,0]
    a2c_gt1  = test_label[i,:,:,0,0]

    a2c_seg1_n = a2c_seg1 == 0
    a2c_seg1_t = a2c_seg1 == 1

    a2c_gt1_n = a2c_gt1 == 0
    a2c_gt1_t = a2c_gt1 == 1


    a2c_tp = np.sum(a2c_seg1_t&a2c_gt1_t)
    a2c_fp = np.sum(a2c_seg1_t&a2c_gt1_n)
    a2c_tn = np.sum(a2c_seg1_n&a2c_gt1_n)
    a2c_fn = np.sum(a2c_gt1_t&a2c_seg1_n)

    a2c_dice = (2*a2c_tp) / (2*a2c_tp + a2c_fp + a2c_fn)

    if a2c_dice !=0 and (2*a2c_tp + a2c_fp + a2c_fn)!=0:
        
        a2c_dice_list.append(a2c_dice)
        a2c_jac_list.append(a2c_dice / (2 - a2c_dice))

############### A4C ################

    a4c_seg1 = pred_label[i,:,:,0,1]
    a4c_gt1  = test_label[i,:,:,0,1]

    a4c_seg1_n = a4c_seg1 == 0
    a4c_seg1_t = a4c_seg1 == 1

    a4c_gt1_n = a4c_gt1 == 0
    a4c_gt1_t = a4c_gt1 == 1


    a4c_tp = np.sum(a4c_seg1_t&a4c_gt1_t)
    a4c_fp = np.sum(a4c_seg1_t&a4c_gt1_n)
    a4c_tn = np.sum(a4c_seg1_n&a4c_gt1_n)
    a4c_fn = np.sum(a4c_gt1_t&a4c_seg1_n)


    a4c_dice = (2*a4c_tp) / (2*a4c_tp + a4c_fp + a4c_fn)

    if a4c_dice !=0 and (2*a4c_tp + a4c_fp + a4c_fn)!=0 :
        
        a4c_dice_list.append(a4c_dice)
        a4c_jac_list.append(a4c_dice / (2 - a4c_dice))
            

print('-'*40)
print('A2C')
print('-'*40)
print('DSC: ', np.mean(a2c_dice_list))
print('Jaccard: ', np.mean(a2c_jac_list))

print('-'*40)
print('A4C')
print('-'*40)
print('DSC: ', np.mean(a4c_dice_list))
print('Jaccard: ', np.mean(a4c_jac_list))

----------------------------------------
A2C
----------------------------------------
DSC:  0.9694793173253917
Jaccard:  0.94272953644586
----------------------------------------
A4C
----------------------------------------
DSC:  0.981010448543785
Jaccard:  0.9632704817360096
